In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs_min = "20250620"
day_obs_min = "yesterday"
day_obs_max = "today"
#day_obs_max = "2024-12-10"
#instrument = "latiss"  
#instrument = "lsstcomcam"
instrument = "lsstcam"

# ConsDB Visit Metadata from {{ params.day_obs_min }} to {{ params.day_obs_max }}

In [ ]:
import os
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import logging

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

In [ ]:
import os
if os.getenv("EXTERNAL_INSTANCE_URL") is not None:
    print("updating rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

from rubin_nights import connections
import rubin_nights.dayobs_utils as rn_dayobs

In [ ]:
if isinstance(day_obs_min, str):
    if day_obs_min.lower() == "today":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_min.lower() == "yesterday":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())
if isinstance(day_obs_max, str):
    if day_obs_max.lower() == "today":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_max.lower() == "yesterday":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())

try:
    t_start = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format='isot', scale='utc')
except ValueError:
    print(f"Is day_obs_min the right format? {day_obs_min} should be YYYYMMDD")
    t_start = None
try:
    t_end = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
except ValueError:
    print(f"Is day_obs_max the right format? {day_obs_max} should be YYYYMMDD")
    t_start = None

if t_start is None or t_end is None:
    print("Did not get valid inputs for time period.")

In [ ]:
day_min = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format='isot', scale='utc')
day_max = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format='isot', scale='utc')
one_day = TimeDelta(1, format='jd')
days = day_min + one_day * np.arange(0, (day_max - day_min).jd + 1)
day_obs_list = [d.iso[0:10] for d in days]

## Accessing visits from ConsDB (USDF)

In [ ]:
if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Hack for Lynne - but you can use your own token (outside of RSP)
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'

else:
    tokenfile = None
    site = None

endpoints = connections.get_clients(tokenfile, site)
consdb = endpoints['consdb']

In [ ]:
visits = consdb.get_visits(instrument, day_min, day_max + one_day)

if len(visits) > 0:
    display(Markdown(f"Retrieved {len(visits)} visits from consdb from {day_min.iso} to {(day_max + one_day).iso}"))
if len(visits) == 0:
    display(Markdown(f"No visits for {instrument} between {day_obs_min} to {day_obs_max} retrieved from consdb"))

In [ ]:
## Almanac ## 
display(Markdown(f"## Daily information for {day_obs_min} to {day_obs_max}"))
site = Site('LSST')
almanac = Almanac()
alm = {}
if len(visits) > 0:
    for day_obs in day_obs_list:
        alm[day_obs] = {}
        night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
        horizon_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
        horizon_sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
        sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
        sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
        night_length = (sunrise.mjd - sunset.mjd) * 24
        alm[day_obs]['horizon sunset'] = horizon_sunset.iso
        alm[day_obs]['sunset'] = sunset.iso
        alm[day_obs]['sunrise'] = sunrise.iso
        close = horizon_sunrise - TimeDelta(2*60*60, format='sec')
        alm[day_obs]['close'] = close
        night_hours = (close.mjd - sunset.mjd)*24
        if np.isnan(night_events['moonrise']):
            alm[day_obs]['moon rise'] = np.nan
        else:
            alm[day_obs]['moon rise'] = Time(night_events['moonrise'], format='mjd', scale='utc').iso
        if np.isnan(night_events['moonset']):
            alm[day_obs]['moon set'] = np.nan
        else:
            alm[day_obs]['moon set'] = Time(night_events['moonset'], format='mjd', scale='utc').iso
        moon_phase = almanac.get_sun_moon_positions(sunset.mjd)['moon_phase']
        alm[day_obs]['moon phase'] = moon_phase.round(2)
        dayobsint = int(day_obs.replace('-', ''))
        vv = visits.query('day_obs == @dayobsint')
        alm[day_obs]['programs'] = vv.science_program.unique()
        alm[day_obs]['nvisits dayobs'] = len(vv)
        visit_times = visits['exp_midpt']
        vv = visits.query("day_obs == @dayobsint and exp_midpt_mjd >= @sunset.mjd and (img_type.str.contains('science') or img_type.str.contains('acq')) and can_see_sky == True")
        alm[day_obs]['nvisits night'] = len(vv)
        alm[day_obs]['sunset to night visits'] = ((vv.exp_midpt_mjd.min() - sunset.mjd) * 24).round(3)
        alm[day_obs]['night visits to close'] = ((close.mjd - vv.exp_midpt_mjd.max()) * 24).round(3)
        alm[day_obs]['first to last visit @ night'] = round((vv.exp_midpt_mjd.max() - vv.exp_midpt_mjd.min()) * 24, 3)
        alm[day_obs]['night length'] =  night_hours.round(3) # night_length.round(3)
        alm[day_obs]['sci_frac'] = (vv.exp_time.sum() / (night_hours * 60 * 60)).round(3)
    alm = pd.DataFrame(alm)
    print("Night visits are after -12 deg sunset and type 'science'")
    print("Time durations are in hours")
    print("Sunset or rise without qualifier is 12 degree sunset or sunrise")
    print("Night length is -12 degree sunset to administrative close at 2 hours for 0-deg sunrise.")
    print("sci_frac is science exposure time / night length.")
    with option_context('display.max_colwidth', None):
        display(HTML(alm.T.to_html()))

In [ ]:
# plt.plot(alm.columns.values, alm.loc['sci eff'], 'k.')
# _ = plt.xticks(rotation=45)
# _ = plt.ylabel("OpenShutterFraction", fontsize='large')

In [ ]:
c = None
if len(visits) > 0:
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type'], dropna=False).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})

In [ ]:
# This might work .. to help translate test block numbers above into more meaningful programs
testcase_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"
jiraticket_base_url = "https://rubinobs.atlassian.net/browse/"

if len(visits) > 0:
    jira_urls = {}
    for science_program in c.sort_values(by=('visit_id', 'first'))[('science_program', 'first')].values: # visits.science_program.unique():
        if science_program is not None and science_program.startswith("BLOCK-T"):
            jira_urls[science_program] = testcase_base_url + science_program
        elif science_program is not None and science_program.startswith("BLOCK-"):
            jira_urls[science_program] = jiraticket_base_url + science_program
        else:
            jira_urls[science_program] = ''
            #display(Markdown(f"[{science_program}]({jira_url}) - {test_name} ({len(visits.query('science_program == @science_program'))} visits)"))

In [ ]:
if len(visits) > 0:
    jira_url_col = np.array(["X" *  (max([len(v) for v in jira_urls.values()])+100)] * len(c))
    for i, (ri, row) in enumerate(c.iterrows()):
        sp = row[('science_program', 'first')]
        if sp in jira_urls.keys():
            jira_url_col[i] = f'<a href="{jira_urls[sp]}" target="_blank" rel="noreferrer noopener">{sp}</a>'
        else:
            jira_url_col[i] = ''
    c['JIRA'] = jira_url_col

In [ ]:
if len(visits) > 0:
    display(Markdown(f"ConsDB Visits"))
    with option_context('display.max_colwidth', None):
        cols = [('JIRA', ''), ('target_name', 'unique'), ('observation_reason', 'unique'), ('day_obs', 'nunique'),
                ('visit_id', 'first'), ('visit_id', 'last'), ('visit_id', 'count')]
        display(HTML(c[cols].sort_values(by=('visit_id', 'first')).to_html(escape=False)))

## Visits vs Time properties

This is a visits vs time example from schedview, using better Rubin color mapping for the various filters.  Due to memory and size, this runs on the last three nights only. (download notebook and run yourself for more). 

In [ ]:
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    os.environ["SCHEDVIEW_DATASOURCE_URL"] = endpoints['api_base']
    os.environ["SCHEDVIEW_EFD_NAME"] = 'usdf_efd'

import warnings
import bokeh.io

bokeh.io.output_notebook()

logging.getLogger('rubin_sim').setLevel(logging.ERROR)
logging.getLogger('schedview').setLevel(logging.ERROR)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import schedview.plot

from rubin_sim.maf import ObservationStartTimestampStacker

print("Shows only the last 3 nights of the timespan")

# Hack the tooltip - Eric will fix this a different way soon.
def visits_tooltips() -> list:
    deg = "\u00b0"
    tooltips = [
        (
            "Start time",
            "@start_timestamp{%F %T} UTC"
        ),
        ("VisitId", "@visit_id"),
        ("Band", "@band"),
        (
            "Field coordinates",
            "RA=@s_ra" + deg + ", Decl=@s_dec" + deg + ", Az=@azimuth" + deg + ", Alt=@altitude" + deg,
        ),
        ("Target name", "@target_name"),
        ("Observation reason", "@observation_reason"),
        ("Science program", "@science_program"),
        ("Scheduler note", "@scheduler_note"),
    ]
    return tooltips

schedview.plot.visits.visits_tooltips = visits_tooltips

if len(visits):
    timestamp_stacker = ObservationStartTimestampStacker(mjd_col="obs_start_mjd",)
    dayobs_cutoff = int((day_max - TimeDelta(3, format='jd')).iso[0:10].replace('-', ''))
    vv = visits.query('day_obs > @dayobs_cutoff').copy()
    vv = timestamp_stacker.run(vv, override=True)
    if len(vv):
        col = 'fwhm_eff'
        if col not in vv.columns:
            col = 'airmass'
        plot = bokeh.plotting.figure(y_axis_label=col, x_axis_label="Time (UTC)", height=600, width=1200)
        fig = schedview.plot.plot_visit_param_vs_time(vv, col, show_column_selector=True, hovertool=True, plot=plot)
        bokeh.io.show(fig)
else:
    print("No visits")